# Setup

In [30]:
# was needed to download files from repo

# Download TorchVision repo to use some files from
# references/detection
# !git clone https://github.com/pytorch/vision.git
# !cd vision
# !git checkout v0.3.0

# !cp references/detection/utils.py ../
# !cp references/detection/transforms.py ../
# !cp references/detection/coco_eval.py ../
# !cp references/detection/engine.py ../
# !cp references/detection/coco_utils.py ../

# Imports

In [74]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from PIL import Image

import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
from train import train_one_epoch, evaluate
import torchvision
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
import matplotlib.pyplot as plt
import os
import utils
from DatasetLoaders import HRSIDDataset

# loading data into fiftyone

In [ ]:
# Loading Dataset

name = "HRSID"
dataset_dir = os.path.abspath("Datasets/HRSID")

# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset

dataset_traintest = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name=name,
)

In [ ]:
name = "HRSID_train"
dataset_dir = os.path.abspath("Datasets/HRSID")
label_path = os.path.abspath("Datasets/HRSID/annotations/train2017.json")
# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset

dataset_train = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name=name,
    labels_path=label_path
)

 100% |███████████████| 3642/3642 [19.5s elapsed, 0s remaining, 171.2 samples/s]      


In [ ]:
name = "HRSID_test"
dataset_dir = os.path.abspath("Datasets/HRSID")
label_path = os.path.abspath("Datasets/HRSID/annotations/test2017.json")
# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset

dataset_test = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name=name,
    labels_path=label_path
)

 100% |███████████████| 1962/1962 [10.3s elapsed, 0s remaining, 107.5 samples/s]     


In [ ]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

# explore fiftyone useage

In [ ]:
paths = dataset_train.values("filepath")
sample = dataset_train[paths[1]]
sample.metadata

<ImageMetadata: {
    'size_bytes': None,
    'mime_type': None,
    'width': 800,
    'height': 800,
    'num_channels': None,
}>

In [ ]:
#sample

In [53]:
img = Image.open(paths[1]).convert("RGB")
detections = sample["detections"].detections
segmentations = sample["segmentations"].detections

In [54]:
classes = dataset_train.distinct("%s.detections.label" % "segmentations")

In [82]:
segmentations

[<Detection: {
     'id': '643908295d834f7b1f102069',
     'attributes': {},
     'tags': [],
     'label': 'ship',
     'bounding_box': [0.3825, 0.65, 0.01625, 0.045],
     'mask': array([[False,  True,  True,  True, False, False, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True, False, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True,  True, False, False,
             False, False, False, False],
            [ True,  True,  True,  True,  True,  True,  True, False, False,
             False, False, False, False

In [55]:
labels_map_rev = {c: i for i, c in enumerate(classes)}
labels_map_rev


{'ship': 0}

In [83]:
for det in segmentations:
    category_id=labels_map_rev[det.label]
    coco_obj=fouc.COCOObject.from_label(det, sample.metadata, category_id=category_id)
    x,y,w,h=coco_obj.bbox

In [80]:
torch.round(torch.as_tensor([coco_obj.segmentation], dtype=torch.float))

tensor([[[569., 606., 542., 604., 530., 598., 561., 594., 576., 596., 582.,
          602., 569., 606.]]])

In [45]:
#segmentations

In [ ]:
session = fo.launch_app()

# Training ResNet Model

In [75]:
def get_model(num_classes):
    weights = FCN_ResNet50_Weights.DEFAULT
    model = fcn_resnet50(weights=weights, classes=num_classes)

    return model

In [76]:
def do_training(model, torch_dataset, torch_dataset_test, num_epochs=4):
    data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=2, shuffle=True, num_workers=2,
        collate_fn=utils.collate_fn
    )
    data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=1, shuffle=False, num_workers=2,
        collate_fn=utils.collate_fn
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using {device}")
    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params, lr=0.001,
                                momentum=0.9, weight_decay=0.0005)

    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=5,
                                                    gamma=0.1)
    for epoch in range(num_epochs):
        train_one_epoch(model=model,
                        criterion=criterion,
                        optimizer=optimizer, 
                        data_loader=data_loader,
                        lr_scheduler=lr_scheduler,
                        device=device, 
                        epoch=epoch, 
                        print_freq=10)

        evaluate(model=model, data_loader=data_loader_test, device=device)

In [77]:
HRSID_train = HRSIDDataset(dataset_train)
HRSID_test = HRSIDDataset(dataset_test)

In [78]:
model = get_model(2)

In [79]:
do_training(model, HRSID_train, HRSID_test, num_epochs=20)

Using cuda


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/k3vinli/miniconda3/envs/torch-38/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/k3vinli/miniconda3/envs/torch-38/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/k3vinli/miniconda3/envs/torch-38/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/k3vinli/ENEE439/Capstone/DatasetLoaders.py", line 67, in __getitem__
    target["masks"] = torch.round(torch.as_tensor(masks, dtype=torch.float))
RuntimeError: value cannot be converted to type uint8 without overflow
